In [1]:
import numpy as np
import sympy as sp
import galois

Now lets do it for composite systems. We now consider a $p$-dimensional Hilbert space (for $p$ odd), and the tensor product of $e$ such such spaces $H = \mathcal{H} \otimes \cdots \otimes \mathcal{H}$.

In [33]:
p = 3 # odd characteristic only
e = 2
d = p**e
F = galois.GF(d, irreducible_poly='x^2+x+2')
# F = galois.GF(d)
F.elements

GF([0, 1, 2, 3, 4, 5, 6, 7, 8], order=3^2)

In [34]:
F.irreducible_poly

Poly(x^2 + x + 2, GF(3))

In [35]:
print(F.repr_table())

 Power   Polynomial   Vector   Integer 
------- ------------ -------- ---------
   0         0        [0, 0]      0     
  x^0        1        [0, 1]      1     
  x^1        x        [1, 0]      3     
  x^2      2x + 1     [2, 1]      7     
  x^3      2x + 2     [2, 2]      8     
  x^4        2        [0, 2]      2     
  x^5        2x       [2, 0]      6     
  x^6      x + 2      [1, 2]      5     
  x^7      x + 1      [1, 1]      4     


Define the characters $\chi(\alpha)$.

In [36]:
def lift(a):
    return np.int64(np.array(a))

def omega(a):
    return sp.exp(sp.I * 2 * sp.pi * lift(a) / p)

def chi(a):
    return omega(a.field_trace())

In [37]:
chi(F.elements[1] + F.elements[2]) == chi(F.elements[1]) * chi(F.elements[2]) # sanity check

True

We are working with $\mathcal{H} \otimes \mathcal{H}$.

In [38]:
for a in F.elements:
    print(a, a.vector())

0 [0 0]
1 [0 1]
2 [0 2]
3 [1 0]
4 [1 1]
5 [1 2]
6 [2 0]
7 [2 1]
8 [2 2]


Field element $m$ with index 6 has vector representation $(0,2)$ (in Vourdas notation), therefore the position state is given by
$$|X; m\rangle = |X; 0\rangle \otimes |X; 2\rangle.$$
This element corresponds to `sp.eye(p)[:,0]` $\otimes$ `sp.eye(p)[:,2]` as opposed to `sp.eye(d)[:,6]`.

In [39]:
sp.tensorproduct(sp.eye(p)[:,0], sp.eye(p)[:,2]).reshape(d,1)

[[0], [0], [1], [0], [0], [0], [0], [0], [0]]

In [40]:
def posx(m):
    # v = np.flip(np.array(m.vector()))
    v = np.array(m.vector()) # should be flipped according to Vourdas!
    x = sp.eye(1)
    for j in v:
        x = sp.tensorproduct(x, sp.eye(p)[:, j])
    return sp.Matrix(x.reshape(d, 1))

In [41]:
for m in F.elements:
    print(m, m.vector(), posx(m))

0 [0 0] Matrix([[1], [0], [0], [0], [0], [0], [0], [0], [0]])
1 [0 1] Matrix([[0], [1], [0], [0], [0], [0], [0], [0], [0]])
2 [0 2] Matrix([[0], [0], [1], [0], [0], [0], [0], [0], [0]])
3 [1 0] Matrix([[0], [0], [0], [1], [0], [0], [0], [0], [0]])
4 [1 1] Matrix([[0], [0], [0], [0], [1], [0], [0], [0], [0]])
5 [1 2] Matrix([[0], [0], [0], [0], [0], [1], [0], [0], [0]])
6 [2 0] Matrix([[0], [0], [0], [0], [0], [0], [1], [0], [0]])
7 [2 1] Matrix([[0], [0], [0], [0], [0], [0], [0], [1], [0]])
8 [2 2] Matrix([[0], [0], [0], [0], [0], [0], [0], [0], [1]])


In [42]:
def tensorproduct(a, b):
    return sp.ImmutableMatrix((sp.tensorproduct(a, b)).reshape(d, d))

In [43]:
FF = sp.zeros(d, d)
for m in F.elements:
    for n in F.elements:
        FF += chi(m * n) * tensorproduct(posx(m), sp.adjoint(posx(n)))
FF = FF / sp.sqrt(d)

In [44]:
(FF * sp.adjoint(FF)).applyfunc(sp.nsimplify) # sanity check

Matrix([
[1, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 1, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 1, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 1, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 1]])

Now we define the displacement operators. This is done directly, without the prime displacement operators.

In [45]:
def Z(a):
    z = sp.zeros(d, d)
    for n in F.elements:
        z += chi(a * n) * tensorproduct(posx(n), sp.adjoint(posx(n)))
    return z

def X(b):
    x = sp.zeros(d, d)
    for n in F.elements:
        p = FF * posx(n)
        x += chi(-(b * n)) * tensorproduct(p, sp.adjoint(p))
    return x.applyfunc(sp.nsimplify)

In [46]:
# sanity check
for a in F.elements:
    # if not (Z(a) * Z(a) == Z(a + a)):
    if not (X(a) * X(a) == X(a + a)):
        raise Exception('Property does not hold!')
print('Property holds!')

Property holds!


In [47]:
X(F.elements[1]) * posx(F.elements[0])

Matrix([
[0],
[1],
[0],
[0],
[0],
[0],
[0],
[0],
[0]])

In [48]:
def D(a, b):
    return chi(-(F.elements[2]**-1) * a*b) * Z(a) * X(b)

In [59]:
D(F.elements[0], F.elements[1])

Matrix([
[0, 0, 1, 0, 0, 0, 0, 0, 0],
[1, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 1, 0, 0, 0],
[0, 0, 0, 1, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 1],
[0, 0, 0, 0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 1, 0]])

In [50]:
b = F.elements[4]
P = sp.zeros(d, d)
for a in F.elements:
    P += D(a,b)
P = P / d
# 'marginal property' check
P.applyfunc(sp.nsimplify) == tensorproduct(posx(F.elements[2]**-1 * b), sp.adjoint(posx(-F.elements[2]**-1 * b)))

True

Now we define the displaced parity operators. At the origin we have $P(0,0) = F^2$, for arbitrary points we have
$$P(\alpha,\beta) = D(\alpha,\beta) P(0,0) D(\alpha,\beta)^* = D(2\alpha,2\beta)P(0,0).$$

In [51]:
def P(a, b):
    return (D(2*a, 2*b) * FF**2).applyfunc(sp.nsimplify)

In [52]:
P(F.elements[2], F.elements[1])**2 # sanity check

Matrix([
[1, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 1, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 1, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 1, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 1]])

In [53]:
b = F.elements[6]
proj = sp.zeros(d, d)
for a in F.elements:
    proj += P(a,b)
proj = proj / d
# 'marginal property' check
proj.applyfunc(sp.nsimplify) == tensorproduct(posx(b), sp.adjoint(posx(b)))

True

The displaced parity operators are enough to define the Wigner function for primer power dimension in the case $p > 2$. The Wigner function is defined for density operator $\rho$ as
$$W(\alpha,\beta) = \text{Tr}[\rho D(\alpha,\beta)].$$

In [54]:
def W(rho):
    w = sp.zeros(d, d)
    for i, a in enumerate(F.elements):
        for j, b in enumerate(F.elements):
            w[i,j] = sp.trace(rho * P(a, b))
    return w

In [55]:
S = sp.Matrix(tensorproduct(posx(F.elements[2]), sp.adjoint(posx(F.elements[2]))))

In [56]:
W(S)

Matrix([
[0, 0, 1, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0, 0, 0, 0]])

In [57]:
v = FF * posx(F.elements[2])
S = sp.Matrix(tensorproduct(v, sp.adjoint(v)))
W(S).applyfunc(sp.nsimplify)

Matrix([
[0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0],
[1, 1, 1, 1, 1, 1, 1, 1, 1],
[0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0]])

Comparison using Vourdas' example ($d = 3$). It's the same!

In [31]:
# state = (sp.eye(d)[:,0] + sp.I*sp.eye(d)[:,1] + 2*sp.eye(d)[:,2])/sp.sqrt(6)
# S = tensorproduct(state, sp.adjoint(state))
# W(S).applyfunc(sp.nsimplify).applyfunc(sp.N)

Matrix([
[ 0.166666666666667,  0.833333333333333, 0.666666666666667],
[-0.410683602522959, -0.166666666666667,  0.95534180126148],
[ 0.744016935856292, -0.166666666666667, 0.377991532071854]])

Let's now use Vourdas' method to calculate the MUBs. We must calculate the symplectic transformations in the position basis.